In [13]:
library(shiny)
library(ggplot2)
library(DT)
library(rsconnect)

In [2]:
data  <-read.csv("hulu_clean.csv")

In [3]:
data

X,show_name,show_clips_count,show_episodes_count,main_genre,show_rating,show_seasons_count,show_videos_count,show_company_name,Action.and.Adventure,⋯,Romantic.Comedy,Saturday.Morning.Cartoons,Science.Fiction,Science.Fiction...Thriller,Sitcoms,Sketch.Comedy,Soap.Operas,Talk.and.Interview,Teen,Videogames
0,family-guy,288,288,Animation and Cartoons,4.353707,15,576,FOX,0,⋯,0,0,0,0,1,0,0,0,1,0
1,south-park,3874,274,Comedy,4.363032,20,4148,Comedy Central,0,⋯,0,0,0,0,0,0,0,0,0,0
2,american-dad,121,172,Comedy,4.114007,9,293,Fox Television Classics,0,⋯,0,0,0,0,1,0,0,0,0,0
3,law-and-order-special-victims-unit,176,406,Drama,4.622856,18,582,NBC,0,⋯,0,0,0,0,0,0,0,0,0,0
4,naruto-shippuden,16,597,Anime,4.400626,9,613,Viz Media,1,⋯,0,0,0,0,0,0,0,0,0,0
5,seinfeld,63,171,Comedy,4.366506,9,234,Sony Pictures Television,0,⋯,0,0,0,0,1,0,0,0,0,0
6,modern-family,68,22,Comedy,4.640557,1,90,ABC,0,⋯,0,0,0,0,1,0,0,0,1,0
7,adventure-time,223,223,Kids,4.365803,7,446,Cartoon Network,0,⋯,0,0,0,0,0,0,0,0,1,0
8,saturday-night-live,7073,370,Comedy,4.083795,18,7443,NBC,0,⋯,0,0,0,0,0,1,0,0,0,0
9,the-mindy-project,23,107,Comedy,4.440965,5,130,Hulu Original Series,0,⋯,0,0,0,0,1,0,0,0,0,0


In [4]:
min_clip <- min(data$show_clips_count)
max_clip <- max(data$show_clips_count)
min_ep <- min(data$show_episodes_count)
max_ep <- max(data$show_episodes_count)
min_season <- min(data$show_seasons_count)
max_season <- max(data$show_seasons_count)
min_video <- min(data$show_videos_count)
max_video <- max(data$show_videos_count)

In [5]:
ui <- fluidPage(headerPanel("Hulu Data"),    
        fluidRow(column(6,sliderInput(inputId = "range_clip",label ="Show Clip Count",
                        min=min_clip,max=max_clip,value=c(min_clip,max_clip),step = 1,width = '600px')),
                 column(6,sliderInput(inputId = "range_ep",label ="Show Episode Count",
                        min=min_ep,max=max_ep,value=c(min_ep,max_ep),step = 1,width = '600px')),
                 column(6,sliderInput(inputId = "range_season",label ="Show Season Count",
                        min=min_season,max=max_season,value=c(min_season,max_season),step = 1,width = '600px')),
                 column(6,sliderInput(inputId = "range_video",label ="Show Video Count",
                        min=min_video,max=max_video,value=c(min_video,max_video),step = 1,width = '600px'))
                     ),
        fluidRow(
            tabsetPanel(
            tabPanel("By Genre",
                     navlistPanel(
                        tabPanel("Avg Ratings",plotOutput(outputId='hist_genre')),
                        tabPanel("N of Shows",plotOutput(outputId = 'n_genre')),
                        tabPanel("Data Table",DT::dataTableOutput("table_genre"))
                     )),
            tabPanel("By Media Company",
                     navlistPanel(
                        tabPanel("Avg Ratings",plotOutput(outputId='hist_comp')),
                        tabPanel("N of Shows",plotOutput(outputId = 'n_comp')),
                        tabPanel("Data Table",DT::dataTableOutput("table_comp"))
                     )),
            tabPanel("Histograms for Filters",
                    navlistPanel(
                        tabPanel("Show Clip Count",plotOutput(outputId = "histo_1")),
                        tabPanel("Show Episode Count",plotOutput(outputId = "histo_2")),
                        tabPanel("Show Season Count",plotOutput(outputId = "histo_3")),
                        tabPanel("Show Video Count",plotOutput(outputId = "histo_4"))
                    ))
            )
            )
        )

In [6]:
server <- function(input,output) {
    data2 <- reactive({
        a <- subset(data, (show_clips_count>=input$range_clip[1]&
                           show_clips_count<=input$range_clip[2]&
                           show_episodes_count>=input$range_ep[1]&
                           show_episodes_count<=input$range_ep[2]&
                           show_seasons_count>=input$range_season[1]&
                           show_seasons_count<=input$range_season[2]&
                           show_videos_count>=input$range_video[1]&
                           show_videos_count<=input$range_video[2]
                          ))
        return(a)})
#   vis for comp
    output$hist_comp <- renderPlot({
        ggplot(data2(), aes(x=show_company_name, y=show_rating,fill=show_company_name))+
        stat_summary(fun.y="mean", geom="bar") +
        theme(axis.text.x = element_text(angle = 90, hjust = 1))
    })
    
    output$n_comp <- renderPlot({
        ggplot(data2(),aes(x=show_company_name)) + 
        geom_bar(aes(fill=show_company_name)) + 
        theme(axis.text.x = element_text(angle = 90, hjust = 1))
    })
    
    output$table_comp <- DT::renderDataTable(DT::datatable(data2()))

#   viz for genre
    output$hist_genre <- renderPlot({
        ggplot(data2(), aes(x=main_genre, y=show_rating,fill=main_genre))+
        stat_summary(fun.y="mean", geom="bar") +
        theme(axis.text.x = element_text(angle = 90, hjust = 1))
    })
    
    output$n_genre <- renderPlot({
        ggplot(data2(),aes(x=main_genre)) + 
        geom_bar(aes(fill=main_genre)) + 
        theme(axis.text.x = element_text(angle = 90, hjust = 1))
    })
    
    output$table_genre <- DT::renderDataTable(DT::datatable(data2()))
    
#     histograms
    output$histo_1 <- renderPlot({
        ggplot(data2(),aes(x = show_clips_count))+
        geom_histogram()+
        geom_density(col=2) + 
        labs(x="Show Clip Count", y="# of shows")
    })
    
    output$histo_2 <- renderPlot({
        ggplot(data2(),aes(show_episodes_count))+
        geom_histogram() + 
        geom_density(col=2) + 
        labs(x="Show Episdoe Count", y="# of shows")
    })
    
    output$histo_3 <- renderPlot({
        ggplot(data2(),aes(show_seasons_count))+
        geom_histogram() + 
        geom_density(col=2) + 
        labs(x="Show Season Count", y="# of shows")
    })
    
    output$histo_4 <- renderPlot({
        ggplot(data2(),aes(show_videos_count))+
        geom_histogram() + 
        geom_density(col=2) + 
        labs(x="Show Video Count", y="# of shows")
    })

    }

In [ ]:
# load
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:4142
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.
`stat_bin()` using `bins =

In [ ]:
# create histogram for data set

In [14]:
# deploy app
# rsconnect::deployApp('/Users/patrick.chen/Desktop/shiny-server/hulu')

Preparing to deploy application...DONE
Uploading bundle for application: 231174...DONE
Deploying bundle: 1055053 for application: 231174 ...
Waiting for task: 494052774
  building: Building image: 1054623
  building: Fetching packages
  building: Installing packages
  building: Installing files
  building: Pushing image: 1054623
  deploying: Starting instances
  rollforward: Activating new instances
  terminating: Stopping old instances
Application successfully deployed to https://pchen2890.shinyapps.io/hulu/
